Helper Functions

In [1]:
from nltk.stem import PorterStemmer

def StemWords(doc_list):
    porter = PorterStemmer()
    new_word_list = ""
    newDocList = []
    for doc in doc_list:
        for word in doc.split(' '):
            new_word_list += porter.stem(word) + ' ' 
        newDocList.append(new_word_list)
        
    return newDocList

In [12]:
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize


def removestopwords(doc_list):
    newDocList = []
    for doc in doc_list:
        pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
        newDocList.append(re.sub(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*', '', doc))
    
    
    return newDocList

In [3]:
import re

def RemoveNonAlpha(doc_list):
    result = []
    for document in doc_list:
        result.append(re.sub(r'([^a-zA-Z \.]+?)', '', document.lower()))
    return result

In [4]:
import csv
import os
import json
from collections import defaultdict
import nltk
import datetime
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords
import re

def GetArticleFromDate(startDate, endDate):

    cord_uid_to_text = defaultdict(list)


    with open('metadata.csv') as f_in:
        reader = csv.DictReader(f_in)
        for row in reader:           
            if '-' not in row['publish_time']:
                continue
            elif startDate > datetime.datetime.strptime(row['publish_time'], '%Y-%m-%d').date() or datetime.datetime.strptime(row['publish_time'], '%Y-%m-%d').date() > endDate:
                continue
            cord_uid = row['cord_uid']
            title = row['title']
            abstract = row['abstract']
            authors = row['authors'].split('; ')

            introduction = []
            if row['pdf_json_files']:
                for json_path in row['pdf_json_files'].split('; '):
                    with open(json_path) as f_json:
                        full_text_dict = json.load(f_json)
                    
                        for paragraph_dict in full_text_dict['body_text']:
                            paragraph_text = paragraph_dict['text']
                            section_name = paragraph_dict['section']
                            introduction.append(paragraph_text)

                        if introduction:
                            break

            cord_uid_to_text[cord_uid].append(introduction)
    
  
    cleanedUpDocList = []
    originalDocList= []
    for key, value in cord_uid_to_text.items():
        doc = ''
        for v in value:
            for m in v:
                doc += m
                #doc += StemWords(removestopwords(re.sub (r'([^a-zA-Z \.]+?)', '', m.lower())))
        if doc:
            cleanedUpDocList.append(doc)
    
    print(len(cleanedUpDocList))
    
    return originalDocList

    

In [5]:
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def ShowKMeansElbowGraph(corpus, clusterRange, startDate, endDate, embedding):
    #corpus = GetArticleFromDate(startDate, endDate)
    embedder = SentenceTransformer(embedding)#'distilbert-base-nli-stsb-mean-tokens'

    corpus_embeddings = embedder.encode(corpus)
    
    ######
    #vectorizer = TfidfVectorizer()
    #corpus_embeddings = vectorizer.fit_transform(corpus)
    #print(corpus_embeddings[0])

    nc = range(1,clusterRange)
    kmeans = []
    score = []
    kmeans = [KMeans(n_clusters=i, init='k-means++', n_init=1000, max_iter=30000, tol=0.0000001)for i in nc]               
    score = [kmeans[i].fit(corpus_embeddings).score(corpus_embeddings) for i in range(len(kmeans))]   
    # Plot the elbow
    plt.plot(nc,score)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Score')
    plt.title('Elbow Curve')
    plt.show()

In [6]:
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from datetime import datetime
from sklearn.cluster import KMeans
from datetime import datetime
import numpy as np

def CalculateAndShowKMeansGraph(num_clusters, startDate, endDate, embedding):
    corpus = GetArticleFromDate(startDate, endDate)
    embedder = SentenceTransformer(embedding)

    corpus_embeddings = embedder.encode(corpus)

    clustering_model = KMeans(n_clusters=i, init='k-means++', n_init=1000, max_iter=30000, tol=0.0000001)
    
    clustering_model.fit(corpus_embeddings)
    # Get the cluster id assigned to each news headline.
    cluster_assignment = clustering_model.labels_ 
    
    cluster_centers = clustering_model.cluster_centers_

    plt.figure(1)
    plt.clf()

    colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
    for k, col in zip(range(num_clusters), colors):
        my_members = cluster_assignment == k
        cluster_center = cluster_centers[k]
        plt.plot(corpus_embeddings[my_members, 0], corpus_embeddings[my_members, 1], col + '.')
        plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
                 markeredgecolor='k', markersize=14)
    plt.title('Estimated number of clusters: %d' % num_clusters)
    plt.show()

    return clustering_model, cluster_centers, corpus_embeddings, corpus
    

Test case 1
No preprocessing on text data

In [7]:
import datetime 
print(datetime.datetime.now())
startDate = datetime.date(2020, 3, 1)
endDate = datetime.date(2020, 4, 1)

ShowKMeansElbowGraph(GetArticleFromDate(startDate, endDate), 10, startDate, endDate,'LaBSE')
print(datetime.datetime.now())

2021-06-16 18:46:28.809846
3413


KeyboardInterrupt: 

Test case 2
Removing stop words only

In [13]:
import datetime 
print(datetime.datetime.now())
startDate = datetime.date(2020, 3, 1)
endDate = datetime.date(2020, 4, 1)

articles = removestopwords(GetArticleFromDate(startDate, endDate))

print(len(articles))
ShowKMeansElbowGraph(articles, 10, startDate, endDate,'LaBSE')
print(datetime.datetime.now())

2021-06-16 18:51:34.609443
3413
0


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

Test case 3
Stem words

Test case 4
Lam words

Test case 5
Remove stop words and stem

Test case 6
Remove stop words and lam 

Test case 7
Remove stem and lam

Test case 8
Remove non-alpha characters

Test case 9
Remove non-alpha and stop words

Test case 10
Remove non-alpha and stem

Test case 11
Remove non-alpha and lam

Test case 12
Remove non-alpha, stop words, and stem

Test case 13
Remove non-alpha, stop words, and lam

Test case 14
Remove non-alpha, stop words, stem, and lam

Test case 15
Lower case words

Test case 16
lower case words and remove stop words

Test case 17
lower case words and stem

Test case 18
lower case words and lam

Test case 19
lower case words and remove non-alpha

Test case 20
lower case words, remove stop words, stem

Test case 21
lower case words, remove stop words, lem

Test case 22
lower case words, remove stop words, remove non-alpha

Test case 23
lower case words, remove stop words, remove non-alpha, stem

Test case 24
lower case words, remove stop words, remove non-alpha, lam

Test case 25
lower case words, remove stop words, remove non-alpha, stem and lam

Look into normalizing text